In [11]:
import win32com.client
import os
import pandas as pd

In [12]:
# Create a DataFrame to store the email log
email_log_filepath = os.path.abspath(r"D:\GIKI\CAA_intern\email_log_with_attachments.xlsx")

 Here, the email log and attachments log are loaded from the existing Excel file if it exists. If the file doesn't exist, new empty logs are created. The last serial numbers from the existing logs are used to increment the serial counters for new entries.

In [13]:
# Check if the email log file already exists
if os.path.exists(email_log_filepath):
    # Load the existing email log from the file
    email_log = pd.read_excel(email_log_filepath, sheet_name='EmailLog')
    attachments_log = pd.read_excel(email_log_filepath, sheet_name='AttachmentsLog')
    
    # Get the last serial number from the existing logs
    last_email_serial = email_log['EmailSerial'].max()
    last_attachment_serial = attachments_log['AttachmentSerial'].max()
    
    # Increment the serial counters for new entries
    email_serial_counter = last_email_serial + 1
    attachment_serial_counter = last_attachment_serial + 1
else:
    # Create new email log and attachments log if the file doesn't exist
    email_log = pd.DataFrame(columns=['EmailSerial', 'Sender', 'Receiver', 'Time', 'Subject'])
    attachments_log = pd.DataFrame(columns=['EmailSerial', 'AttachmentSerial', 'FileType', 'RenamedAttachment'])
    
    # Initialize the serial counters
    email_serial_counter = 1
    attachment_serial_counter = 1


In [14]:
# Access Outlook and retrieve the inbox folder
outlook = win32com.client.Dispatch("Outlook.Application")
mapi = outlook.GetNamespace("MAPI")
inbox = mapi.GetDefaultFolder(6)  # Inbox folder
messages = inbox.Items

In [15]:
# Create a folder to save the attachments
attachments_folder = r"D:\GIKI\CAA_intern\attachments"
os.makedirs(attachments_folder, exist_ok=True)

In [16]:
# Iterate over unread emails, download attachments, mark as read, and update the log
for message in messages:
    if message.Unread:
        sender = message.SenderEmailAddress
        receiver = message.ReceivedByName
        time = message.ReceivedTime
        subject = message.Subject
        
        # Add email information to the email log
        email_log = email_log.append({
            'EmailSerial': email_serial_counter,
            'Sender': sender,
            'Receiver': receiver,
            'Time': str(time),
            'Subject': subject
        }, ignore_index=True)

        attachments_info = []  # List to store attachment information

        for attachment in message.Attachments:
            filename = attachment.FileName
            file_extension = os.path.splitext(filename)[1]
            new_filename = f"{email_serial_counter}_{attachment_serial_counter}_{filename}"
            filepath = os.path.abspath(os.path.join(attachments_folder, new_filename))
            attachment.SaveAsFile(filepath)

            attachments_info.append({
                'EmailSerial': email_serial_counter,
                'AttachmentSerial': attachment_serial_counter,
                'FileType': file_extension,
                'RenamedAttachment': new_filename
            })

            attachment_serial_counter += 1

        # Mark the email as read
        message.UnRead = False

        # Add attachment information to the attachment log
        for attachment_info in attachments_info:
            attachments_log = attachments_log.append(attachment_info, ignore_index=True)

        email_serial_counter += 1

C:\Users\Owaise\AppData\Local\Temp\ipykernel_5888\3737421670.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  email_log = email_log.append({
C:\Users\Owaise\AppData\Local\Temp\ipykernel_5888\3737421670.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  attachments_log = attachments_log.append(attachment_info, ignore_index=True)
C:\Users\Owaise\AppData\Local\Temp\ipykernel_5888\3737421670.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  attachments_log = attachments_log.append(attachment_info, ignore_index=True)


In [25]:
email_log.head(10)

,EmailSerial,Sender,Receiver,Time,Subject
0,1,/O=EXCHANGELABS/OU=EXCHANGE ADMINISTRATIVE GRO...,u2020337,2023-06-27 13:31:57.139000+00:00,HEC - Coursera: Cohort I
1,2,owais.sajid002@gmail.com,u2020337,2023-06-23 11:25:04.207000+00:00,Attachment test 3


In [33]:
email_log['Sender']

0    /O=EXCHANGELABS/OU=EXCHANGE ADMINISTRATIVE GRO...
1                             owais.sajid002@gmail.com
Name: Sender, dtype: object

In [26]:
attachments_log.head(10)

,EmailSerial,AttachmentSerial,FileType,RenamedAttachment
0,1,1,.jpg,1_1_HEC-DLSE Cohort I.jpg
1,2,2,.docx,2_2_CV_assignment3.docx
2,2,3,.png,2_3_fiverr-logo.png


In [17]:
# Save the updated email log and attachments log to the same Excel file
with pd.ExcelWriter(email_log_filepath) as writer:
    email_log.to_excel(writer, sheet_name='EmailLog', index=False)
    attachments_log.to_excel(writer, sheet_name='AttachmentsLog', index=False)

### Sorting into seperate folders..

In [18]:
import os
import shutil

The separate folders for each extension will be created within the attachments_folder itself. 

The function will create a folder for each unique extension encountered and move the files with that extension into their respective folders within the attachments_folder.

In [19]:
def sort_attachments_by_extension(attachments_folder):
    # Get all files in the attachments folder
    files = os.listdir(attachments_folder)

    # Create a dictionary to store folders for each extension
    extension_folders = {}

    # Iterate over the files and identify their extensions
    for file in files:
        file_path = os.path.join(attachments_folder, file)
        
        # Exclude folders and files within extension folders from sorting 
        # ' .isfile() ' functions checks to make sure that it is a file ! 
        if not os.path.isfile(file_path) or os.path.dirname(file_path) != attachments_folder:  
            continue # Sorting restricted to main "attachments_folder" only.

        file_extension = os.path.splitext(file)[1]

        # Create a folder for the extension if it doesn't exist
        if file_extension not in extension_folders:
            extension_folder = os.path.join(attachments_folder, f"{file_extension}_folder") # making of sub-folders
            os.makedirs(extension_folder, exist_ok=True)
            extension_folders[file_extension] = extension_folder

        # Move the file to the respective extension folder
        destination_folder = extension_folders[file_extension]
        destination_path = os.path.join(destination_folder, file)
        shutil.move(file_path, destination_path)

In [20]:
sort_attachments_by_extension(attachments_folder)